<a href="https://colab.research.google.com/github/benderminguez89/ArtificialIntelligence/blob/main/Copy_of_571_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
spamOrHam_df = pd.read_csv('/content/gdrive/MyDrive/AI/SpamDetection.csv')
#display first 5 rows of dataframe
spamOrHam_df.head()

,Target,data
0,spam,WINNER As a valued network customer you have b...
1,spam,If you had your mobile for more than an year y...
2,ham,I am gonna be home soon
3,spam,SIX chances to win CASH From 100 to 20000 doll...
4,spam,URGENT You have won a 1 week FREE membership i...


In [ ]:
#TASK 1:
#spliting dataframe into training and test sets
train_df = spamOrHam_df[0:20]
test_df = spamOrHam_df[20:]
#displaying info about each dataframe
print(f"training set:\n{train_df.info()}")
print(f"test set:\n{test_df.info()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Target  20 non-null     object
 1   data    20 non-null     object
dtypes: object(2)
memory usage: 448.0+ bytes
training set:
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 20 to 29
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Target  10 non-null     object
 1   data    10 non-null     object
dtypes: object(2)
memory usage: 292.0+ bytes
test set:
None


In [ ]:
#TASK 2:
#for train_df, display values from the "Target" column
print(train_df['Target'].value_counts())
#probabilities: P(ham), P(spam)
p_spamPrior = 9/20
p_hamPrior = 11/20
#display calculated probabilities
print(f"\nthe prior probability of ham is: \n\tP(ham) = {p_hamPrior}\nthe prior probability of spam is: \n\tP(spam) = {p_spamPrior}\n")
#display normalized values to confirm calculated probabilities
print(train_df['Target'].value_counts(normalize=True))

ham     11
spam     9
Name: Target, dtype: int64

the prior probability of ham is: 
	P(ham) = 0.55
the prior probability of spam is: 
	P(spam) = 0.45

ham     0.55
spam    0.45
Name: Target, dtype: float64


In [ ]:
sentence_list = []
word_list = []
#for each sentence in the training data set
for i in train_df['data']:
  #print(i)
  #add sentence to the sentence_list
  sentence_list.append(i)
  #create list of words in each sentence, split at ' '(white space)
  words = i.split()
  #for each word in list of words
  for w in words:
    #print(w)
    word_list.append(w)
#create a dictionary with key:value pair of unique words:count
word_dict = {word:word_list.count(word) for word in word_list}
#print(f'{sentence_list}\n{word_dict}')

In [ ]:
#split train_df into two sub dataframes for each of the Target categories
spam_df = train_df[train_df['Target']=='spam']
ham_df = train_df[train_df['Target']=='ham']
print(f'HAM:\n{ham_df}\nSPAM:\n{spam_df}')

spam_sentences = []
spam_words = []
ham_sentences = []
ham_words = []
spam_w_count = 0
ham_w_count = 0
for i in spam_df['data']:
  #print(i)
  spam_sentences.append(i)
  sw = i.split()
  for s in sw:
    spam_w_count += 1
    spam_words.append(s)
  

for j in ham_df['data']:
  ham_sentences.append(j)
  hw = j.split()
  for h in hw:
    ham_w_count += 1
    ham_words.append(h)
  

HAM:
   Target                                               data
2     ham                           I am gonna be home soon 
5     ham  You have been wonderful and a blessing at all ...
7     ham                            okay I am watching here
9     ham          Is that seriously how you spell his name 
10    ham                  did you finish your lunch already
11    ham       Alright no way I can meet up with you sooner
12    ham  Just forced myself to eat a slice I am really ...
13    ham                            Did you catch the train
14    ham                        tell me anything about you.
16    ham          Hello How are you and how did saturday go
19    ham                 Sorry I will call later in meeting
SPAM:
   Target                                               data
0    spam  WINNER As a valued network customer you have b...
1    spam  If you had your mobile for more than an year y...
3    spam  SIX chances to win CASH From 100 to 20000 doll...
4    spam  UR

In [ ]:
spam_dict = {word:spam_words.count(word) for word in spam_words}
print(f'{spam_sentences}\n{spam_dict}\n{len(spam_dict)}\n{len(spam_words)}')

ham_dict = {word:ham_words.count(word) for word in ham_words}
print(f'\n{ham_sentences}\n{ham_dict}')

['WINNER As a valued network customer you have been selected to receive a prize reward To claim call ', 'If you had your mobile for more than an year you are entitled to Update to the latest colour mobiles with camera for Free Call The Mobile ', 'SIX chances to win CASH From 100 to 20000 dollars text or call', 'URGENT You have won a 1 week FREE membership in our million dollar Prize Jackpot Text the word CLAIM ', 'XXXMobileMovieClub To use your credit click the WAP link in the next txt message or click here ', 'England vs Macedonia dont miss the team news Text ur national team ', 'Thanks for your subscription to Ringtone UK your mobile will be charged Please confirm by replying YES or NO ', 'Rodger Burns MSG We tried to call you reply to our sms for a free samsung mobile and free camcorder Please call now for delivery tomorrow', 'Congrats one year special cinema pass for two is yours call now Dont miss out ']
{'WINNER': 1, 'As': 1, 'a': 4, 'valued': 1, 'network': 1, 'customer': 1, 'you

In [ ]:
#TASK 3:
posterior_probs = {}
for x in word_dict:
  lamplace = False
  tempLamp = 0
  spamPostProb = 0
  hamPostProb = 0
  
  if spam_dict.get(x) is None:
    #print(f"\tP({x}|spam): {0/spam_w_count}\nPerforming Laplace smoothing...")
    #print(f"spam_{x}: {(tempLamp + 1)/(spam_w_count + unique_words)}")
    spamPostProb = (tempLamp + 1)/(spam_w_count + len(spam_dict))
    #print(f"ham_{x}: {(ham_dict.get(x) + 1)/(ham_w_count + unique_words)}")
    hamPostProb = (ham_dict.get(x) + 1)/(ham_w_count + len(ham_dict))
  elif ham_dict.get(x) is None:
    #print(f"\tP({x}|ham): {0/ham_w_count}\nPerforming Laplace smoothing...")
    #print(f"spam_{x}: {(spam_dict.get(x) + 1)/(spam_w_count + unique_words)}")
    spamPostProb = (spam_dict.get(x) + 1)/(spam_w_count + len(spam_dict))
    #print(f"ham_{x}: {(tempLamp + 1)/(ham_w_count + unique_words)}")
    hamPostProb = (tempLamp + 1)/(ham_w_count + len(ham_dict))
  else:
    #print(f"spam_{x}: {spam_dict.get(x)/spam_w_count}")
    spamPostProb = (spam_dict.get(x) + 1)/(spam_w_count + len(spam_dict))
    #print(f"ham_{x}: {ham_dict.get(x)/ham_w_count}")
    hamPostProb = (ham_dict.get(x) + 1)/(ham_w_count + len(ham_dict))
  posterior_probs.update({x:[spamPostProb, hamPostProb]})
  print(f"p({x}|spam) = {posterior_probs.get(x)[0]}\np({x}|ham) = {posterior_probs.get(x)[1]}\n")

p(WINNER|spam) = 0.006968641114982578
p(WINNER|ham) = 0.006578947368421052

p(As|spam) = 0.006968641114982578
p(As|ham) = 0.006578947368421052

p(a|spam) = 0.017421602787456445
p(a|ham) = 0.019736842105263157

p(valued|spam) = 0.006968641114982578
p(valued|ham) = 0.006578947368421052

p(network|spam) = 0.006968641114982578
p(network|ham) = 0.006578947368421052

p(customer|spam) = 0.006968641114982578
p(customer|ham) = 0.006578947368421052

p(you|spam) = 0.017421602787456445
p(you|ham) = 0.039473684210526314

p(have|spam) = 0.010452961672473868
p(have|ham) = 0.013157894736842105

p(been|spam) = 0.006968641114982578
p(been|ham) = 0.013157894736842105

p(selected|spam) = 0.006968641114982578
p(selected|ham) = 0.006578947368421052

p(to|spam) = 0.0313588850174216
p(to|ham) = 0.013157894736842105

p(receive|spam) = 0.006968641114982578
p(receive|ham) = 0.006578947368421052

p(prize|spam) = 0.006968641114982578
p(prize|ham) = 0.006578947368421052

p(reward|spam) = 0.006968641114982578
p(rewa

In [ ]:
#TASK 4:
for sentence in sentence_list:
  prob_sentence_spam = 1
  prob_sentence_ham = 1 
  for word in sentence:
    prob_sentence_ham *= posterior_probs.get(x)[1]
    prob_sentence_spam *= posterior_probs.get(x)[0]
    #print(f"spam:{prob_sentence_spam}, ham{prob_sentence_ham}")
  spamClass = prob_sentence_spam * p_spamPrior
  hamClass = prob_sentence_ham * p_hamPrior
  if spamClass >= hamClass:
    print(f"SPAM: Confidence = {spamClass}")
  else:
    print(f"HAM: Confidence = {hamClass}")

HAM: Confidence = 3.4659081851109746e-187
HAM: Confidence = 1.542009164197107e-260
HAM: Confidence = 3.9887468548248675e-46
HAM: Confidence = 1.3487143182184738e-117
HAM: Confidence = 6.000533561480219e-191
HAM: Confidence = 6.5897582604952e-97
HAM: Confidence = 1.1563023887151295e-179
HAM: Confidence = 3.031447609666899e-44
HAM: Confidence = 5.319269651113417e-127
HAM: Confidence = 4.2364823947693314e-78
HAM: Confidence = 4.715352281818832e-63
HAM: Confidence = 9.650829190592038e-84
HAM: Confidence = 2.5989715279012846e-106
HAM: Confidence = 3.031447609666899e-44
HAM: Confidence = 9.086480479171678e-52
HAM: Confidence = 5.391146466500668e-206
HAM: Confidence = 4.2364823947693314e-78
HAM: Confidence = 1.1719269647898013e-258
HAM: Confidence = 1.088685632662727e-147
HAM: Confidence = 6.204410897130041e-65


In [ ]:
def classify(sentence):
  prob_sentence_spam = 1
  prob_sentence_ham = 1 
  for word in sentence:
    prob_sentence_ham *= posterior_probs.get(x)[1]
    prob_sentence_spam *= posterior_probs.get(x)[0]
    #print(f"spam:{prob_sentence_spam}, ham{prob_sentence_ham}")
  spamClass = prob_sentence_spam * p_spamPrior
  hamClass = prob_sentence_ham * p_hamPrior
  if spamClass >= hamClass:
    print(f"SPAM: Confidence = {spamClass}")
    return "spam"
  else:
    print(f"HAM: Confidence = {hamClass}")
    return "ham"

In [ ]:
correctlyPredicted = 0
for i in range(len(train_df)):
  print(f"{train_df.iloc[i, 1]}")
  result = classify(train_df.iloc[i, 1])
  print(f"\n\tpredicted as: {result}")
  if train_df.iloc[i, 0] == result:
    correctlyPredicted += 1

WINNER As a valued network customer you have been selected to receive a prize reward To claim call 
HAM: Confidence = 3.4659081851109746e-187

	predicted as: ham
If you had your mobile for more than an year you are entitled to Update to the latest colour mobiles with camera for Free Call The Mobile 
HAM: Confidence = 1.542009164197107e-260

	predicted as: ham
I am gonna be home soon 
HAM: Confidence = 3.9887468548248675e-46

	predicted as: ham
SIX chances to win CASH From 100 to 20000 dollars text or call
HAM: Confidence = 1.3487143182184738e-117

	predicted as: ham
URGENT You have won a 1 week FREE membership in our million dollar Prize Jackpot Text the word CLAIM 
HAM: Confidence = 6.000533561480219e-191

	predicted as: ham
You have been wonderful and a blessing at all times
HAM: Confidence = 6.5897582604952e-97

	predicted as: ham
XXXMobileMovieClub To use your credit click the WAP link in the next txt message or click here 
HAM: Confidence = 1.1563023887151295e-179

	predicted as: 

In [ ]:
#TASK 5:
correctlyPredicted = 0
for i in range(len(test_df)):
  print(f"{test_df.iloc[i, 1]}")
  result = classify(test_df.iloc[i, 1])
  print(f"predicted as: {result}")
  if test_df.iloc[i, 0] == result:
    correctlyPredicted += 1

print(f"Accuracy = {correctlyPredicted/len(test_df)}")

Tell where you reached
HAM: Confidence = 2.3039001833468436e-42
predicted as: ham
Your gonna have to pick up a burger for yourself on your way home
HAM: Confidence = 3.07241015048311e-123
predicted as: ham
As a valued customer I am pleased to advise you that for your recent review you are awarded a Bonus Prize
HAM: Confidence = 1.7986037725717344e-198
predicted as: ham
Urgent you are awarded a complimentary trip to EuroDisinc To claim text immediately
HAM: Confidence = 4.293728009854186e-157
predicted as: ham
Finished class where are you
HAM: Confidence = 1.1955895367331156e-53
predicted as: ham
where are you how did you perform 
HAM: Confidence = 6.204410897130041e-65
predicted as: ham
you can call me now
HAM: Confidence = 1.0113568868848642e-36
predicted as: ham
I am waiting Call me once you are free
HAM: Confidence = 1.8597140957262622e-72
predicted as: ham
I am on the way to homei
HAM: Confidence = 3.9887468548248675e-46
predicted as: ham
Please call our customer service representa